In [1]:
import SimFunctions
import SimRNG
import SimClasses
import numpy as np
import pandas as pd

In [4]:
class MG1():
    def __init__(self,MeanTBA = 1.0, MeanST = 0.8, Phases=3, RunLength = 550000.0, WarmUp = 5000.0):
        """
        MeanTBA := MeanTimeBetweenArrival
        MeanST := MeanServiceTime
        RunLength := The size of the simulation
        WarmUp := Data will be ignored in WarmUp period,
        this is to capture more data close to steady-state of
        the simulation
        """
        
        self.MeanTBA = MeanTBA
        self.MeanST = MeanST
        self.RunLength = RunLength
        self.WarmUp = WarmUp
        self.Phases = Phases
        
        self.Queue = SimClasses.FIFOQueue()
        self.Wait = SimClasses.DTStat()
        self.Server = SimClasses.Resource()
        self.Calendar = SimClasses.EventCalendar()
        
        self.AllWaitMean = []
        self.AllQueueMean = []
        self.AllQueueNum = []
        self.AllServerMean = []
        
        self.TheCTStats = []
        self.TheDTStats = []
        self.TheQueues = []
        self.TheResources = []
        
        self.TheDTStats.append(self.Wait)
        self.TheQueues.append(self.Queue)
        self.TheResources.append(self.Server)
        self.Server.SetUnits(1)
    def Arrival(self):
        """
        function schedule the arrival of a customer, if server is
        empty it assign it to the server and schedule the end
        of service time.
        """
        SimFunctions.Schedule(self.Calendar, 'Arrival', SimRNG.Expon(
        self.MeanTBA, 1))
        Customer = SimClasses.Entity()
        self.Queue.Add(Customer)
        if self.Server.Busy == 0:
            self.Server.Seize(1)
            self.Server
            SimFunctions.Schedule(self.Calendar,"EndOfService",SimRNG.Erlang(self.Phases,self.MeanST,2))
    def EndOfService(self):
        """
        Empties up the server, then, if queue is non-empty, schedule
        another service.
        """
        DepartingCustomer = self.Queue.Remove()
        self.Wait.Record(SimClasses.Clock - DepartingCustomer.CreateTime)
        if self.Queue.NumQueue() > 0:
            SimFunctions.Schedule(self.Calendar, 'EndOfService', SimRNG.Erlang(self.Phases,self.MeanST,2))
            self.Server.Seize(1)
        else:
            self.Server.Free(1)
    def Stats(self, reps = 10):
        """
        Simulate MG1 for the given parameters over the number of
        reps. 
        """
        for i in range(reps):

            """Initializing the Calendar, specifically indicating
            the EndSimulation time and WarmUp time"""
            SimFunctions.SimFunctionsInit(self.Calendar, self.TheQueues,
                                self.TheCTStats,
                                self.TheDTStats,
                                self.TheResources)
            SimFunctions.Schedule(self.Calendar, 'Arrival', SimRNG.
                                  Expon(self.MeanTBA,1))
            SimFunctions.Schedule(self.Calendar, 'EndSimulation', self.RunLength)
            
            SimFunctions.Schedule(self.Calendar, "ClearIt", self.WarmUp)
            
            
            """
            Initializing the first event
    
            """
            NextEvent = self.Calendar.Remove()
            SimClasses.Clock = NextEvent.EventTime
            
            if NextEvent.EventType == 'Arrival':
                self.Arrival()
            elif NextEvent.EventType == 'EndOfService':
                self.EndOfService()
            elif NextEvent.EventType == "ClearIt":
                SimFunctions.ClearStats(self.TheCTStats,self.TheDTStats)
            while NextEvent.EventType !='EndSimulation':
                NextEvent = self.Calendar.Remove()
                SimClasses.Clock = NextEvent.EventTime
            
                if NextEvent.EventType == 'Arrival':
                    self.Arrival()
                elif NextEvent.EventType == 'EndOfService':
                    self.EndOfService()
                elif NextEvent.EventType == "ClearIt":
                    SimFunctions.ClearStats(self.TheCTStats,self.TheDTStats)
            
            self.AllWaitMean.append(self.Wait.Mean())
            self.AllQueueMean.append(self.Queue.Mean())
            self.AllQueueNum.append(self.Queue.NumQueue())
            self.AllServerMean.append(self.Server.Mean())
            print (i+1, self.Wait.Mean(), self.Queue.Mean(), self.Queue.NumQueue(), self.Server.Mean())


In [6]:
sim = MG1()
sim.Stats(10)

1 2.9330348633898935 2.934901440446175 2 0.8013098113142109
2 2.9407825109162866 2.9399599685373743 0 0.8005266237391743
3 2.9213769788505983 2.921497915154473 2 0.8003648091542754
4 2.943130749181187 2.943845108942107 1 0.7997681322515211
5 2.9324641955838526 2.926611685169056 6 0.7992721906839975
6 2.8910601016922777 2.8942668476040088 6 0.7999428341496183
7 2.9694299669107544 2.9739460908942497 1 0.8022115028023167
8 2.9442132790238573 2.9459968149964797 6 0.8009133449514262
9 2.9161727915401388 2.910241860078334 5 0.7980951100258326
10 2.908881828987184 2.909901872562175 1 0.8002071156982747


1 2.897370866603768 2.8810661013144028 2 0.7958935810405309
2 2.7906708203938084 2.7657842032054822 1 0.7921983937133946
3 2.9610372143285013 2.9598022672305793 5 0.7994499202437041
4 2.811565770886649 2.7960073054896037 1 0.7941261543049896
5 2.8909290604642597 2.8801193930176705 0 0.8003105823520983
6 3.1540045103600716 3.183523686450747 11 0.8101449089932978
7 2.982188564472856 3.0180458059918416 5 0.8063949639030745
8 2.7927970859964284 2.7821844570696452 0 0.7961052154470782
9 2.9256634600994493 2.9264271440863725 1 0.8026326279327308
10 3.060268542044961 3.0527402694690755 4 0.8027835372923164
